In [ ]:
import numpy as np
import pandas as pd
import Amazon_Movie_Parser as amp
import Graph_Amazon_Movies as gam
# save numpy array as csv file
from numpy import savetxt
# load numpy array from csv file
from numpy import loadtxt
import networkx as nx
import datetime
import Computations_debug as cmp
from networkx.algorithms import bipartite
import random
import Predict_Movie_v1 as pm
import pickle
from sklearn.metrics import mean_squared_error
import os

In [ ]:
#import matplotlib.pylot as plt

## Normalization: TILE (broadcast) or MATRIX (np.newaxis) (FASTER)

In [ ]:
# Normalization test with TILE
tt = np.array([[2, 4, 6], [1, 5, 8], [20, 4, 6]])
#print(tt.mean(axis=1))
#tt_norm=tt/tt.mean(axis=1)

print('Sample matrix:\n{}'.format(tt))

tt_sum = tt.sum(axis = 1)
print('\nSum of each row:\n{}'.format(tt.sum(axis=1)))

# broadcast with TILE so that we can divide the each element in the row with the same number 
# we will do an element wise division not a dot product here
tt_sum_broadcasted = np.tile(tt_sum, (3, 1)).T
print('\nBroadcasting with TILE:\n{}'.format(tt_sum_broadcasted))

# divide each element in the row with the sum of the row so the sum will be equal to 1
# in other words, it will give the probability/rating for the each cell
tt_norm = tt / tt_sum_broadcasted
print('\nNormalizing with TILE:\n{}'.format(tt_norm))


In [ ]:
# Here, we are testing the normalization using the prepared function, to see if it works as expected
cmp1 = cmp.GraphComp()
tt = np.array([[2, 4, 6], [1, 5, 8], [20, 4, 6]])

#print('Sample matrix:\n{}'.format(tt))

# TILE put in function Normalize_Array()
tt_norm = cmp1.Normalize_Array(tt, axis = 1)
print('\nNormalizing with Normalize_Array function:\n{}'.format(tt_norm))

In [ ]:
# Here, we are testing the normalization using the prepared function, to see if it works as expected
cmp1 = cmp.GraphComp()
tt = np.array([[2, 4, 6], [1, 5, 8], [20, 4, 6]])

#print('Sample matrix:\n{}'.format(tt))

#Normalize_Matrix(self, arr_to_normalize, dim = 'row'):
tt_norm = cmp1.Normalize_Matrix(tt, dim = 'tt')
print('\nNormalizing with Normalize_Matrix function:\n{}'.format(tt_norm))

## Selecting Random Numbers: choices vs sample (UNIQUE)

In [ ]:
tt = [2, 4, 6, 8, 9, 11]
# random.choices 
# random.sample returns unique numbers unlike random.choices 
random.sample(tt, k = 2)

## Convert from Graph to Matrix using Networkx (FASTER than FOR LOOP)

In [ ]:
# Let's see normalization from our dataset to verify that it is working fine
# below that one, you will find a 3x3 matrix simulation, if it would be easier to understand, you can check that one first..
grpComp = cmp.GraphComp()
grp = gam.Graph_Amazon()

# here, there is no Connected or Disconnected check, so, the graph generated will most likele be a DISCONNECTED graph
DISconnected_gr_amazon_movies = grp.Create_Graph(file_name = 'data/movies.txt', n_movies = 8, prs_out = 'dictionary')
# here, we set n_movies = 8 but we have 10 nodes. Why is that?
# Because 8 actually shows the number of edges, not number of the movies (BAD NAMING!)
# So, we have 8 edges and in this 8 edges, we have 10 unique nodes. It might be like 8 users + 2 movies

P_disconnected = nx.to_numpy_matrix(DISconnected_gr_amazon_movies)
print(type(P_disconnected))

print('P_disconnected:\n{}'.format(P_disconnected))

P_disconnected_norm = grpComp.Normalize_Matrix(P_disconnected, dim = 'row')
print('\nP_disconnected-NORMALIZED:\n{}'.format(P_disconnected_norm))

In [ ]:
## Another simulation with a smaller matrix, it migh
tt2 = np.array([[0, 4, 2], [4, 0, 0], [2, 0, 0]])
tt2 = np.asmatrix(tt2)
print(type(tt2))
#tt2_norm = cmp1.Normalize_Matrix(tt2, dim = 'tt2')
tt2_norm = cmp1.Normalize_Matrix(tt2, dim = 'row')

print('Sample matrix:\n{}'.format(tt2))
print('\nSample matrix_NORM:\n{}'.format(tt2_norm))

r_tt = np.zeros(tt2_norm.shape[0])
r_tt[1] = 1
r_tt_zero = r_tt.copy()

print('r at the beginning:\n{}'.format(r_tt_zero))

print('\nLets see how random walk works by selecting the second node_P[1]:'.format(r_tt))
#r = (1 - beta) * np.dot(r, P_norm) + beta * r_zero

# r vectors can be generated in the function but we started to design in that way, sos didn't change it later..
r_tt = grpComp.random_walk_vector(tt2_norm, r_tt, r_tt_zero, beta=0.15, n_steps=1)
print('r - after 1st step:\n{}'.format(r_tt))
# as you can see, after the first strep, the value for r[1]=0.15  and the SUM(row) = 1

r_tt = grpComp.random_walk_vector(tt2_norm, r_tt, r_tt_zero, beta=0.15, n_steps=1)
print('\nr - after 2nd step:\n{}'.format(r_tt))

r_tt = grpComp.random_walk_vector(tt2_norm, r_tt, r_tt_zero, beta=0.15, n_steps=1)
print('\nr - after 3rd step:\n{}'.format(r_tt))

r_tt = grpComp.random_walk_vector(tt2_norm, r_tt, r_tt_zero, beta=0.15, n_steps=1)
print('\nr - after 4th step:\n{}'.format(r_tt))

r_tt = grpComp.random_walk_vector(tt2_norm, r_tt, r_tt_zero, beta=0.15, n_steps=1)
print('\nr - after 5th step:\n{}'.format(r_tt))

# You will easily realize that, for the even number of steps, the probability is high on the user himself,
# on the odd number of steps, the probability is high on the movie he reviewed

In [ ]:
grpComp = cmp.GraphComp()

# *** min numner of reviews is 140 
# if we want to make a connected bipartite graph & if we start picking from review#1
grp = gam.Graph_Amazon()
file_name='data/movies.txt'; n_movies = 140; prs_out='dictionary'

max_connected_gr_amazon_movies = grpComp.Create_Bipartite_Giant_Component(grp, file_name, n_movies, prs_out)

P = nx.to_numpy_matrix(max_connected_gr_amazon_movies)

grpComp.Normalize_Matrix(P, dim = 'row')


## Random Walk Simulation

In [ ]:
r = np.zeros(P_disconnected_norm.shape[0])
r[1] = 1
r_zero = r.copy()

print('r at the beginning:\n{}'.format(r_zero))

print('\nLets see how random walk works by selecting the second node_P[1]:'.format(r))
#r = (1 - beta) * np.dot(r, P_norm) + beta * r_zero

r = grpComp.random_walk_vector(P_disconnected_norm, r, r_zero, beta=0.15, n_steps=1)
print('\nr - after 1st step:\n{}'.format(r))
# as you can see, after the first strep, the value for r[1]=0.15  and the SUM(row) = 1

r = grpComp.random_walk_vector(P_disconnected_norm, r, r_zero, beta=0.15, n_steps=1)
print('\nr - after 2nd step:\n{}'.format(r))

r = grpComp.random_walk_vector(P_disconnected_norm, r, r_zero, beta=0.15, n_steps=1)
print('\nr - after 3rd step:\n{}'.format(r))

## Find Common items in 2 lists with SET (FASTER) - WITHOUT FOR loop

In [ ]:
bottom_nodes = ['u1', 'u2', 'u3', 'u4']
bottom_nodes_v = ['u_v1', 'u2', 'u_v3', 'u4', 'u_v5']
lst_common_users = list(set(bottom_nodes) - (set(bottom_nodes) - set(bottom_nodes_v)))
#lst_common_users = list(set(bottom_nodes_v) - (set(bottom_nodes_v) - set(bottom_nodes)))
print(lst_common_users)

## Create Graph from LIST

In [ ]:
lst_test = [('A141HP4LYPW', 'B003AI2VGA', '3.0'),
     ('B000063W1R', 'A141HP4LYPW', '4.0'),
     ('6304286961', 'A141HP4LYPW', '5.0'),
     ('A141HP4LYPW', '5556167281', '5.0')]

grp1 = gam.Graph_Amazon()
grp_test=grp1.Create_Graph_From_List_WITH_Weight(lst_test)

print(grp_test.nodes)

grp_test_martrix = nx.to_numpy_matrix(grp_test)

print(grp_test_martrix)
# as uyou can see, we have 1 user 4 movies in this set (which is unusual, movies are usually less)
# but this is just an artificial set - you can easily play and test some functions if you would like to now
# i.e. Normalization, Random walk, check networkx functions, etc...

In [ ]:
grp_test.edges.data('weight', default=1)

In [ ]:
grp_test.nodes

In [ ]:
list(grp_test.nodes)[1]

In [ ]:
grp_test.has_edge('A141HP4LYPW', 'B003AI2VGAx')

In [ ]:
gdata = grp_test.get_edge_data('A141HP4LYPW', 'B003AI2VGA')
print(gdata)

In [ ]:
gdata['weight']

In [ ]:
grp_test.get_edge_data('A141HP4LYPW', 'B003AI2VGA')['weight']

In [ ]:
type(grp_test.get_edge_data('A141HP4LYPW', 'B003AI2VGAx'))

In [ ]:
# THIS WAS AN INTERNAL TEST - just keeping as a side note..
# by doing the below conversion, we are just considering the links between the nodes, we ignore the weights
# we try to understand if considering the weights will benefit us or not
#P[P>0]=1

In [ ]:
np.argsort(llt)[0][0][0]

In [ ]:
sorted_nodes=np.argsort(r)

user_similarity = []

# we will not consider the user himself or movies as similar nodes
for idx in sorted_nodes[0][0]:
    if idx != 3 and idx > 0:
        user_similarity.append(idx)
        
user_similarity_top2 = user_similarity[-2:]

print(user_similarity)
print(user_similarity_top2)
print(list(grp_test.nodes)[user_similarity_top2[0]])

## Read Edges from movies file, OUTPUT: Screen

In [ ]:
# the code was quickly written so, it might not be necessarily starting from the 4th review but maybe 5th review
# sensitive calibration haven't done, so index can be +1. 
# but in any case, it gurantees to avoid the overlap (if unlucky, max 1 overlap will be in place)
file_name = 'data/movies.txt'; start_index = 4; n_movies = 5; prs_out = 'screen'

#movie_dict_old = amp.AmazonMovies(n_movies, file_name, prs_out)
movie_dict_old = amp.AmazonMovies_VALIDATION(n_movies, file_name, prs_out, start_index)

In [ ]:

file_name = 'data/movies.txt'; start_index = 2; n_movies = 10; prs_out = 'screen'

movie_dict = amp.AmazonMovies_VALIDATION(n_movies, file_name, prs_out, start_index)

## Create test graphs

In [ ]:
grp = gam.Graph_Amazon()
grp_test = grp.Create_Bipartite_VALIDATION(file_name = 'data/movies.txt', start_index = 0, 
                            n_movies = 12000, prs_out = 'dictionary', file_type = 'txt')

In [ ]:
start_time_part1 = datetime.datetime.now()
diameter_of_graph = nx.diameter(grp_test)
end_time_part1 = datetime.datetime.now()
calc_time=end_time_part1-start_time_part1
print(diameter_of_graph)
print('calc time= {}'.format(calc_time))
# diameter for graphs with the edges:
# 12K = 13 .... takes time: 13:24 minutes
# 20K = 13 .... takes time: 35:45 minutes (35 minutes, 45 seconds)
# We will not calculate anymore since it looks like it is independent of the size 
# of the graph and in average, it ends up as 13. Moreover, can we use this diameter 
# function on a bipartite graph is another question***
# 30K =  .... takes time: 
# 40K =  .... takes time:  
# 50K =  .... takes time:  
# 55K =  .... takes time: 

## SEARCH in movies.txt for specific items (FILTER)

In [ ]:
import Amazon_Movie_Parser as amp
import datetime
start_time_part1 = datetime.datetime.now()
file_name = 'data/movies.txt'; n_movies=500000; prs_out = 'dictionary'; num_limit=5000000; date_year=2009

#userID = '';   movieID = ''  #movieID = 'B00005952Q'
#userID = 'AR15V2ULA2EYM'   #1182729600
#pd.set_option('display.max_colwidth', 2000)
#tst_dict = amp.Filter_AmazonMovies(n_movies, userID, movieID, file_name, outputTo='dictionary')
tst_dict = amp.AmazonMovies(num_limit, n_movies, date_year, file_name, prs_out)

end_time_part1 = datetime.datetime.now()
calc_time=end_time_part1-start_time_part1

print('calc time= {}'.format(calc_time))
#amp.Filter_AmazonMovies_2(n_movies, userID, movieID, file_name, prs_out)

i=1274406 .. ts=2012-09-11 .. num_rec=141600
i=1925250 .. ts=2004-12-19 .. num_rec=177757
i=9206322 .. ts=2012-09-11 .. num_rec=582260
i=9863772 .. ts=2012-09-11 .. num_rec=618784
i=18642876 .. ts=2009-09-16 .. num_rec=1106511
i=19191660 .. ts=2009-09-16 .. num_rec=1136998
i=25368990 .. ts=2004-07-21 .. num_rec=1480182
i=25472220 .. ts=2004-12-19 .. num_rec=1485916
i=26664306 .. ts=2008-06-02 .. num_rec=1552142
i=29832738 .. ts=2012-10-18 .. num_rec=1728165
i=31718418 .. ts=2009-09-16 .. num_rec=1832924
i=32248176 .. ts=2012-09-11 .. num_rec=1862354
i=34394424 .. ts=2012-09-13 .. num_rec=1981589
i=38462460 .. ts=2010-09-01 .. num_rec=2207590
i=38720346 .. ts=2012-09-11 .. num_rec=2221916
i=39821136 .. ts=2004-12-19 .. num_rec=2283070
i=48661800 .. ts=2004-09-08 .. num_rec=2774217
i=48661809 .. ts=2004-09-08 .. num_rec=2774217
i=52843524 .. ts=2007-09-05 .. num_rec=3006534
i=53280654 .. ts=2000-12-13 .. num_rec=3030818
i=54416256 .. ts=2012-09-11 .. num_rec=3093906
i=55813722 .. ts=2009

In [10]:
import pandas as pd

lst_movie = []
lst_user = []
lst_rating = []
lst_time = []    
            
for review in tst_dict.values():
    #ts = int(review[2])
    #ts_datetime = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    #ts_datetime = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

    lst_movie.append(review[0])
    lst_user.append(review[1])
    lst_time.append(review[2])
    #lst_time.append(ts_datetime)
    lst_rating.append(review[3])
                
dict_reviews = {'users': lst_user, 'movies': lst_movie, 'rating': lst_rating, 'time': lst_time}

df_reviews = pd.DataFrame(dict_reviews).sort_values('time')
df_reviews = df_reviews.reset_index(drop=True)


In [22]:
df_reviews.head(10)

#duration = 1  # seconds
#freq = 440  # Hz
#os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

In [19]:
df_reviews.query("time < '2009-03-01' and time >= '2009-02-01'")

,users,movies,rating,time
5625,ABLY9PHJR8LQ3,B003UMW648,5.0,2009-02-01
5626,A1YV7JR4Z1G600,B0009IXRIA,5.0,2009-02-01
5627,A3IYMY6QNAVWBM,B001KNAI2Q,5.0,2009-02-01
5628,A37OOQQLMMKYR8,B00006L9XJ,4.0,2009-02-01
5629,AT99KGZL5IZD5,B0010X73ZG,5.0,2009-02-01
...,...,...,...,...
9658,A25ZAY25GR5CFR,B000FDF78W,4.0,2009-02-28
9659,A1LR6W5Q4U32ZW,6300183238,3.0,2009-02-28
9660,A181XRGXAO6COB,B000N4SHOE,4.0,2009-02-28
9661,A2PWMLTJ8L3IYD,B001F8YPLG,2.0,2009-02-28


In [23]:
os.system('say "your program has finished"')

0

In [ ]:
#str_date = "2012-10-25"

int_date=1182729600
dt_date=datetime.utcfromtimestamp(int_date).strftime('%Y-%m-%d')
#ts = datetime.date(*(int(s) for s in dt_date.split('-')))
#ts
dt_date.year

In [ ]:
import Amazon_Movie_Parser as amp
import datetime
# there is a problem with the format at i=1274406 .. ts=2012-09-11 .. num_rec=141600
start_date=datetime.datetime.now()
file_name = 'data/movies.txt'; num_limit=141600; n_movies=50000; prs_out = 'dictionary'; date_year=2009
dict_test=amp.AmazonMovies_DATE(num_limit, n_movies, date_year, file_name, prs_out)
end_date=datetime.datetime.now()
print('Calculartion time: {}'.format(end_date-start_date))
dict_test

In [ ]:
i=3
x=i+9-i%9+1
x%9

In [ ]:
import Amazon_Movie_Parser as amp
from datetime import datetime
file_name = 'data/movies.txt'; num_limit=20; n_movies=50000; prs_out = 'screen'; date_year=2009
amp.AmazonMovies_VALIDATION_DEBUG(num_limit, file_name, prs_out, start_index = 141605)

In [ ]:
12743146%9
12743137

In [ ]:
# the search uses 'like' criteria so it search for any string mathcing 
# not a perfect search, just used to debug something -can definitely be improved..
import Amazon_Movie_Parser as amp
from datetime import datetime
file_name = 'data/movies.txt'; n_movies=100; prs_out = 'screen'
userID = '';   movieID = ''  #movieID = 'B00005952Q'
#userID = 'AR15V2ULA2EYM'   #1182729600
#pd.set_option('display.max_colwidth', 2000)
#tst_dict = amp.Filter_AmazonMovies(n_movies, userID, movieID, file_name, outputTo='dictionary')
review_dict = amp.AmazonMovies(n_movies, file_name, outputTo='dictionary')
list(review_dict.values())

lst_movie = []
lst_user = []
lst_rating = []
lst_time = []

for review in review_dict.values():
    ts = int(review[2])
    #ts_datetime = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    ts_datetime = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')
    
    lst_movie.append(review[0])
    lst_user.append(review[1])
    #lst_time.append(review[2])
    lst_time.append(ts_datetime)
    lst_rating.append(review[3])
    #print('ts={} ts_datetime={}'.format(ts, ts_datetime))
    #print(review)
    
dict_reviews = {'users': lst_user, 'movies': lst_movie, 'rating': lst_rating, 'time': lst_time}

df_reviews = pd.DataFrame(dict_reviews).sort_values('time')#, inplace=True
print(df_reviews)

#ts = 1012003200
#ts_datetime = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
#print(ts_datetime)
# 2000-07-30 00:00:00  ... 964915200
# 2002-01-26 00:00:00  ... 1012003200

In [ ]:
df_time_filtered = df_reviews.query("time > '2012-07-30'")
# if you want to re-index the indexes
df_time_filtered=df_time_filtered.reset_index(drop=True)
#print(df_time_filtered[df_time_filtered.index >= 2 and df_time_filtered.index < 4])
#print(df_time_filtered.query("index >= 2 and index <4"))
n_movies=3
n_movies_val=2
df_reviews_model = df_time_filtered.query("index < " + str(n_movies))
print(df_reviews_model)
df_reviews_val = df_time_filtered.query("index >= " + str(n_movies) + " and index < " + str(n_movies + n_movies_val))
print(df_reviews_val)

In [ ]:
for r in df_time_filtered.values:
    print(r[2])

In [ ]:
df_time_filtered.values

In [ ]:
from datetime import datetime
ts = int("1037491200")
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

import numpy as np

y_true = [0, 0, 0, 0, 1, 1]
y_pred = [0, 0, 1, 0, 0, 1]

fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
lst_tpr = list(tpr); lst_fpr = list(fpr); lst_thresholds = list(thresholds)

print(fpr)
print(tpr)
print(thresholds)

len_tpr = len(lst_tpr)
lst_precision = np.zeros(len_tpr)
lst_recall = np.zeros(len_tpr)
        
for i in range(len(lst_tpr)):
    lst_precision[i] = lst_tpr[i]/(lst_tpr[i] + lst_fpr[i])

roc_auc = roc_auc_score(y_true, y_pred)
print(roc_auc)

precision = precision_score(y_true, y_pred, average='macro') # 'micro'  'weighted' None  average=None, zero_division=1
print(precision)

precision_recall = precision_recall_curve(y_true, y_pred)
print(list(precision_recall))

avg_precision = average_precision_score(y_true, y_pred)
print(avg_precision)

print(lst_sensitivity)

In [ ]:
lst1=[1,2,3]
lst2=[3,5,1]
lst3=[8,2,4]
lstr = [lst1, lst2, lst3]
lstr_2 = []

max_i=0
max_i_val=0
for i, l in enumerate(lstr):
    lstr_2.append(l[2])
    if l[0] > max_i_val:
        max_i_val = l[0]
        max_i = i

print(max_i)
print(max_i_val)
print(lstr[max_i])
print(lstr_2)

In [ ]:
lst4=[8,2,4]
print(lst4)
lst4.remove(8)
print(lst4)

## Mini-Test to Check if any RUN-time Error

In [ ]:
file_name = 'data/movies.txt'; n_movies = 5000; n_movies_v = 3000; n_test_users = 20; walk_steps = 2
start_index_v = n_movies; beta = 0.15; top_neighbor = 10; top_N_movie_suggestions = 5

predict_amazon = pm.Amazon()
strt=datetime.datetime.now()
max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL = \
    predict_amazon.create_graphs(file_name, n_movies, n_movies_v, start_index_v)

df_FINAL = \
    predict_amazon.run_validation_TOP_N(max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL,\
                                  walk_steps, n_test_users, beta, top_neighbor, top_N_movie_suggestions)
end1=datetime.datetime.now()
diff=end1-strt
df_perf = predict_amazon.save_performance_values(df_FINAL, diff, diff)

params_df = predict_amazon.show_parameters_in_use()
params_df.style.set_properties(**{'text-align': 'left'})

df_perf

## OTHERS

In [ ]:
top_N_suggestions=10
predict_amazon.df_summary_ratio_v.sort_values(by='ratio_similar', ascending=False).head(top_N_suggestions)

In [ ]:
file_name = 'data/movies.txt'; num_reviews = 5
amp.AmazonMovies(num_reviews, file_name, outputTo='file')

In [ ]:
lst_tst = amp.Read_Connected_Movies('Movies_22.5.2020_20.47.56.txt')
print(lst_tst)
arr_tst = lst_tst[2][0].replace('[','').replace(']','').split(sep="' '")
#print(len(lst_tst))
print(arr_tst)
print(arr_tst[0])

In [ ]:
#import pickle
file_name = 'data/movies.txt'; num_reviews = 75000

amp.AmazonMovies(num_reviews, file_name, outputTo='dict_to_file')


In [ ]:
xx = amp.Load_Pickle_File_VAL('Movies_22.5.2020_23.21.10__100_reviews.pkl', 10, 5)

In [ ]:
xx

In [ ]:
import pickle
out = dict(list(xx.items())[2: 7])
out

In [ ]:
type(out)

In [ ]:
lst_1=[1]
lst_2=[2,3,1]
lst_1+=lst_2
print(lst_1)

In [ ]:
lst_1.count(2)

## Gathering Results from ALL CSV Files

In [ ]:
file_name = 'RESULTS/_Precision@1/PredictionDetails_50000_20500_100_40_60_TOP1_27.5.2020_18.33.51.csv'
perf_1 = pd.read_csv(file_name)
perf_1

In [ ]:
from sklearn.metrics import mean_squared_error
perf_1['prediction']=1
y_true = list(perf_1['reality'])
#y_pred = list(perf_1['ngbr_ratio'])
y_pred = list(perf_1['prediction'])
mean_squared_error(y_true, y_pred)

In [ ]:
file_name = 'RESULTS/_Precision@1/PerformanceValues_0.903_26.5.2020_15.51.9.csv'
perf_1 = pd.read_csv(file_name)
perf_1

In [ ]:
# Top-K Recommendations Prediction Results
file_name = 'RESULTS/_Precision@1/PerformanceValues_0.903_26.5.2020_15.51.9.csv'
perf_1 = pd.read_csv(file_name)

list_name = ['Test_No', 'AUC_Top_K', 'Precision_Top_K', 'Recall_Top_K', 'n_movies_model', 'n_movies_validation',
            'walk_steps', 'beta', 'top_N_neighbors', 'n_test_users', 'top_K_recommendations']

test_no = 0
AUC = round(float(perf_1["AUC"][0]), 6)
precision = perf_1.query("AUC=='decision_4'")['Precision'].tolist()[0]
recall = perf_1.query("AUC=='decision_4'")['TPR'].tolist()[0]
n_movies_model = int(perf_1["lst_value"][1])
n_movies_validation = int(perf_1["lst_value"][2])
walk_steps = int(perf_1["lst_value"][3])
beta = float(perf_1["lst_value"][4])
top_N_neighbors = int(perf_1["lst_value"][5])
n_test_users = int(perf_1["lst_value"][6])
top_K_recommendations = int(perf_1["lst_value"][10])

list_value = [test_no, AUC, round(precision, 6), round(recall, 6), n_movies_model, n_movies_validation, 
              top_N_neighbors, n_test_users, walk_steps, beta, top_K_recommendations]

list_value

In [ ]:
list(np.random.uniform(0, 1, 4))

In [ ]:
int(perf_1["lst_value"][1])

In [ ]:
import os 
list_file = os.listdir("RESULTS/_Precision@1/")
print(list_file)